In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

C:\Users\causb1a\AppData\Roaming\Python\Python36\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\causb1a\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\causb1a\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\causb1a\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_csv("C:/Users/causb1a/Desktop/smart.csv")

In [4]:
def event_label_cleaner(text):
        """
        cleans the search bar column, where customers have searched. This makes it lowercase and gets rid of stop words,
        punctuation etc. to get it ready for processing
        :param text: the input to be cleaned (string)
        :return: returns a cleaned string output via some processing as above stated.
        """
        text = re.sub('[^a-zA-Z]', ' ', str(text))
        text = re.sub("&lt;/?.*?&gt;", " &lt;&gt; ", text)
        text = re.sub("(\\d|\\W)+", " ", text)
        text = text.lower()
        text = text.split()
        lem = WordNetLemmatizer()
        stop_words = set(stopwords.words("english"))
        text = [lem.lemmatize(word) for word in text if not word in stop_words]
        text = " ".join(text)
        return text

def event_label_cleaner_to_df(df):
    df['clean'] = df['Interaction '].apply(lambda x: event_label_cleaner(x))
    return df

In [5]:
df = event_label_cleaner_to_df(df)

In [6]:
def get_n_gram_df(n_gram_min, n_gram_max, df):
    """
    This function creates a dataframe that counts the number of words that exist. n_gram means number of words in
    combination, for example a bigram means a phrase of 2 words. This function counts how many times that phrase for
    example exists in the specified column of the dataframe. In this case, the clean column after the df has been
    cleaned
    :param n_gram: An integer number of n_gram
    :param call_no_call_df: dataframe with the column search bar to count
    :return: returns a dataframe with a count of each word/phrase
    """
    if df['clean'].empty:
        return pd.DataFrame(columns = ['counts', 'name'])
    vectorizer = CountVectorizer(ngram_range=(n_gram_min, n_gram_max))
    matrix = vectorizer.fit_transform(df.clean)
    words = list(vectorizer.get_feature_names())
    sum_counts = matrix.toarray().sum(axis=0)
    return pd.DataFrame(data={'counts': sum_counts, 'name': words}).sort_values(by='counts', ascending=False)

In [7]:
get_n_gram_df(1, 2, df)

,counts,name
7880,658,ihd
4206,570,customer
2435,465,call
9779,422,meter
1450,389,apr
...,...,...
7041,1,go ahead
7042,1,go back
7043,1,go fixing
7044,1,go high
